In [5]:
# Imports

import os
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from loguru import logger
from pathlib import Path

# Local imports
sys.path.append(r"./utils")
from utils import utils

In [6]:
# Constants

SAMPLE_DATA = True
DEBUG_MODEL = True

GROUP_NAME = "ethnicity_groups"  # TODO : Changed this for ethnicity segments. 
MODEL_DICT = \
            {"NN": {'epochs': 25} , 
             "LR": {'max_iter': 100}
            }
SELF_REPORTED_COLS = \
           ['age',
            # 'country_canada', 
            # 'country_united kingdom',
            # 'country_united states',
            # 'database_dating',
            # 'database_fb',
            # 'gender',
           ]
DATA_DROP_COLS = \
           ['Unnamed: 0', # index columns
            # 'pol',  # label column
            'gender', # self reported and filtered already
            # 'age', # self-reported  
            'country', # self reported and filtered already
            'userid', # index equivalent column 
            'pol_dat_us', # redundant columns with label
            'pol_dat_ca', # redundant columns with label
            'pol_dat_uk', # redundant columns with label
            'pol_fb_us', # redundant columns with label
            'database', # filtered already 
            'ethnicity.value' # filtered already
            ]
RESULTS_COLS = \
            ["Group Name", 
             "Model", 
             "Feature Set", 
             "Test AUC", 
             "Test Accuracy",
            ]
DATA_DIR = "./data/full/"
RESULTS_DIR = f"./results/full/{GROUP_NAME}/" #TODO: ensure that the folder exists
RESULTS_STATS_FILENAME = GROUP_NAME + '.csv'
RESULTS_MODEL_FILENAME_PREFIX = GROUP_NAME

if SAMPLE_DATA:
  DATA_DIR = "./data/sample/"
  RESULTS_DIR = f"./results/sample/{GROUP_NAME}/"
  ASSERT_DATA_SHAPE_0 = 31742
  ASSERT_DATA_SHAPE_1 = 2092
  DATA_DROP_COLS = DATA_DROP_COLS \
                  + ['Unnamed: 0.1'] #TODO: Regenerate sample with index=False and remove this

if DEBUG_MODEL:
  MODEL_DICT = \
            {"NN": {'epochs': 1} , 
             "LR": {'max_iter': 1}
            }

logger.debug(f"Started the script for {GROUP_NAME}.")

2021-08-18 19:00:18.361 | DEBUG    | __main__:<module>:60 - Started the script for ethnicity_groups.


In [7]:
# Read datasets

folders = os.listdir(DATA_DIR)
dataset_paths = []
for folder in tqdm(folders):
  logger.debug(f"In folder {folder}.")
  csv_files = os.listdir(DATA_DIR + folder)
  for csv in csv_files:
    if '.csv' in csv:
      logger.debug(DATA_DIR + folder + "/" + csv)
      dataset_paths.append(DATA_DIR + folder + "/" + csv)


  0%|          | 0/9 [00:00<?, ?it/s]2021-08-18 19:00:18.395 | DEBUG    | __main__:<module>:6 - In folder UK_0_dating.
2021-08-18 19:00:18.396 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_black.csv
2021-08-18 19:00:18.397 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_asian.csv
2021-08-18 19:00:18.398 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_india.csv
2021-08-18 19:00:18.398 | DEBUG    | __main__:<module>:10 - ./data/sample/UK_0_dating/segment_united kingdom_0_dating_white.csv
2021-08-18 19:00:18.398 | DEBUG    | __main__:<module>:6 - In folder Canada_1_dating.
2021-08-18 19:00:18.399 | DEBUG    | __main__:<module>:10 - ./data/sample/Canada_1_dating/segment_canada_1_dating_asian.csv
2021-08-18 19:00:18.399 | DEBUG    | __main__:<module>:10 - ./data/sample/Canada_1_dating/segment_canada_1_dating_india.csv
2021-08-18 19:00:18.400 | DEBUG    

In [8]:
results = []

for dataset_path in dataset_paths:
  data = pd.read_csv(dataset_path)
  
  # Clean the data
  data = data.drop(DATA_DROP_COLS, axis=1)
  data = utils.get_clean_data(data)
  logger.debug(f"Data size after cleaning is {data.shape}")
  data_y = data['pol'].replace({"liberal": 1, "conservative": 0})
  data = data.drop('pol', axis = 1)
  all_features = data.columns

  # Split the data
  X_train, X_test, y_train, y_test = train_test_split(data, data_y, test_size = 0.2, random_state = 2021) 
  del data, data_y

  # which group it is being processed on 
  SEGMENT_NAME = utils.get_dataframe_name(dataset_path)
  logger.debug(f"Started the script for {SEGMENT_NAME}.")
  
  ## Define features for the various settings
  image_cols = list(map(str, range(1, 2049)))
  image_and_self_reported_cols = image_cols + SELF_REPORTED_COLS
  image_and_extracted_cols = [x for x in all_features if x not in SELF_REPORTED_COLS]
  image_and_self_reported_and_extracted_cols = all_features

  data_dict = {
                "Image Features" : image_cols,
                # "Image and Self Reported Features" : image_and_self_reported_cols,
                "Image and Extracted Features": image_and_extracted_cols,
                "Image, Self-reported and Extracted Features": image_and_self_reported_and_extracted_cols
                }

  if SEGMENT_NAME == "canada_1_dating_india": # this segment does not have any "age" values associated with it.
    ## Define features for the various settings
    image_cols = list(map(str, range(1, 2049)))
    # image_and_self_reported_cols = image_cols + SELF_REPORTED_COLS 
    image_and_extracted_cols = [x for x in all_features if x not in SELF_REPORTED_COLS]
    image_and_self_reported_and_extracted_cols = all_features

  
    data_dict = {
                "Image Features" : image_cols,
                # "Image and Self Reported Features" : image_and_self_reported_cols,
                "Image and Extracted Features": image_and_extracted_cols,
                "Image, Self-reported and Extracted Features": image_and_self_reported_and_extracted_cols
                }


  # Fit models and log results
  for data_name, data_set_features in tqdm(data_dict.items()):
    for model_name, model_params in MODEL_DICT.items():
      try:
        save_model_filepath = Path(RESULTS_DIR \
                        + SEGMENT_NAME\
                        + "_" + model_name \
                        + "_" + data_name.replace(" ","_").replace(",","").replace("-","_") \
                        + '.mdl')

        if save_model_filepath.is_file() or save_model_filepath.is_dir():
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model already exists.")
          model = utils.read_model(model_name, save_model_filepath)
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model read from disk.")
          
        else:
          logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model training started.")
          model = utils.fit_model(model_name,
                                X_train[data_set_features],
                                y_train,
                                model_params = model_params)

          utils.save_model(model, model_name, save_model_filepath)
          logger.debug(f"{GROUP_NAME}, {model_name}, {data_name}: model training ended and model saved.")
        
        auc, acc = utils.get_metrics(model_name,
                                     model,
                                     X_test[data_set_features],
                                     y_test)
                                     
        results.append([SEGMENT_NAME, model_name, data_name, auc, acc])
        logger.debug(f"{SEGMENT_NAME}, {model_name}, {data_name}: model training ended. AUC: {auc}, accuracy: {acc}")
      
      except Exception as error:
        logger.exception(error)
        logger.error(f"{SEGMENT_NAME}, {model_name}, {data_name}: Error occured!")

2021-08-18 19:00:19.499 | DEBUG    | __main__:<module>:9 - Data size after cleaning is (1000, 2076)
2021-08-18 19:00:19.505 | DEBUG    | __main__:<module>:20 - Started the script for united kingdom_0_dating_black.
  0%|          | 0/3 [00:00<?, ?it/s]2021-08-18 19:00:19.506 | DEBUG    | __main__:<module>:67 - united kingdom_0_dating_black, NN, Image Features: model training started.


Train on 640 samples, validate on 160 samples
Epoch 1/1
640/640 [==============================] - 1s 947us/step - loss: 0.7042 - accuracy: 0.6125 - val_loss: 0.6359 - val_accuracy: 0.6812


2021-08-18 19:00:20.496 | DEBUG    | __main__:<module>:74 - ethnicity_groups, NN, Image Features: model training ended and model saved.
2021-08-18 19:00:20.610 | DEBUG    | __main__:<module>:82 - united kingdom_0_dating_black, NN, Image Features: model training ended. AUC: 51.74, accuracy: 61.5
2021-08-18 19:00:20.611 | DEBUG    | __main__:<module>:67 - united kingdom_0_dating_black, LR, Image Features: model training started.
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
2021-08-18 19:00:20.731 | DEBUG    | __main__:<module>:74 - ethnicity_groups, LR, Image Features: model training ended and model saved.
2021-08-18 19:00:20.758 | DEBUG    | __main__:<module>:82 - united kingdom_0_dating_black, LR, Image Features: model training ended. AUC: 65.86, accuracy: 62.5
 33%|███▎      | 1/3 [00:01<00:02,  1.25s/it]2021-08

Train on 640 samples, validate on 160 samples
Epoch 1/1
640/640 [==============================] - 1s 947us/step - loss: 1.3851 - accuracy: 0.4125 - val_loss: 3.7979 - val_accuracy: 0.6812


2021-08-18 19:00:21.707 | DEBUG    | __main__:<module>:74 - ethnicity_groups, NN, Image and Extracted Features: model training ended and model saved.
2021-08-18 19:00:21.832 | DEBUG    | __main__:<module>:82 - united kingdom_0_dating_black, NN, Image and Extracted Features: model training ended. AUC: 37.27, accuracy: 61.0
2021-08-18 19:00:21.833 | DEBUG    | __main__:<module>:67 - united kingdom_0_dating_black, LR, Image and Extracted Features: model training started.
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
2021-08-18 19:00:21.964 | DEBUG    | __main__:<module>:74 - ethnicity_groups, LR, Image and Extracted Features: model training ended and model saved.
2021-08-18 19:00:21.988 | DEBUG    | __main__:<module>:82 - united kingdom_0_dating_black, LR, Image and Extracted Features: model training ended. AUC: 52.5

...

2021-08-18 19:00:51.042 | DEBUG    | __main__:<module>:74 - ethnicity_groups, NN, Image and Extracted Features: model training ended and model saved.
2021-08-18 19:00:51.156 | DEBUG    | __main__:<module>:82 - canada_1_dating_black, NN, Image and Extracted Features: model training ended. AUC: 46.47, accuracy: 75.0
2021-08-18 19:00:51.157 | DEBUG    | __main__:<module>:67 - canada_1_dating_black, LR, Image and Extracted Features: model training started.
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
2021-08-18 19:00:51.282 | DEBUG    | __main__:<module>:74 - ethnicity_groups, LR, Image and Extracted Features: model training ended and model saved.
2021-08-18 19:00:51.307 | DEBUG    | __main__:<module>:82 - canada_1_dating_black, LR, Image and Extracted Features: model training ended. AUC: 55.15, accuracy: 75.0
 67%|█

Train on 640 samples, validate on 160 samples
Epoch 1/1
640/640 [==============================] - 1s 959us/step - loss: 0.6917 - accuracy: 0.5297 - val_loss: 2.3305 - val_accuracy: 0.7188


2021-08-18 19:00:52.299 | DEBUG    | __main__:<module>:74 - ethnicity_groups, NN, Image, Self-reported and Extracted Features: model training ended and model saved.
2021-08-18 19:00:52.419 | DEBUG    | __main__:<module>:82 - canada_1_dating_black, NN, Image, Self-reported and Extracted Features: model training ended. AUC: 49.65, accuracy: 75.0
2021-08-18 19:00:52.419 | DEBUG    | __main__:<module>:67 - canada_1_dating_black, LR, Image, Self-reported and Extracted Features: model training started.
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
2021-08-18 19:00:52.542 | DEBUG    | __main__:<module>:74 - ethnicity_groups, LR, Image, Self-reported and Extracted Features: model training ended and model saved.
2021-08-18 19:00:52.565 | DEBUG    | __main__:<module>:82 - canada_1_dating_black, LR, Image, Self-reported and 

Train on 640 samples, validate on 160 samples
Epoch 1/1
640/640 [==============================] - 1s 936us/step - loss: 0.7231 - accuracy: 0.4625 - val_loss: 0.6942 - val_accuracy: 0.5063


2021-08-18 19:00:54.080 | DEBUG    | __main__:<module>:74 - ethnicity_groups, NN, Image Features: model training ended and model saved.


In [ ]:
# Save results summary to disk

save_results_filepath = Path(RESULTS_DIR + RESULTS_STATS_FILENAME)
utils.save_results(results_array = results, 
                     location = save_results_filepath,
                     columns = RESULTS_COLS)
print(pd.DataFrame(results,columns = RESULTS_COLS))
logger.debug(f"Script for {GROUP_NAME} finished.")

2021-08-18 18:55:23.956 | DEBUG    | utils.utils:save_results:263 - Results Saved.
2021-08-18 18:55:23.963 | DEBUG    | __main__:<module>:8 - Script for ethnicity_groups finished.


           Group Name Model                                  Feature Set  \
0    ethnicity_groups    NN                               Image Features   
1    ethnicity_groups    LR                               Image Features   
2    ethnicity_groups    NN             Image and Self Reported Features   
3    ethnicity_groups    LR             Image and Self Reported Features   
4    ethnicity_groups    NN                 Image and Extracted Features   
..                ...   ...                                          ...   
251  ethnicity_groups    LR             Image and Self Reported Features   
252  ethnicity_groups    NN                 Image and Extracted Features   
253  ethnicity_groups    LR                 Image and Extracted Features   
254  ethnicity_groups    NN  Image, Self-reported and Extracted Features   
255  ethnicity_groups    LR  Image, Self-reported and Extracted Features   

     Test AUC  Test Accuracy  
0       58.07           63.5  
1       57.80           5